In [1]:
import numpy as np

In [96]:
def _rotate(inp):
    return np.flip(inp, axis=(0,1))
    
def _convolution_op(inp, kernel, stride=1):
    # inp shape -> 4 dim
    assert len(inp.shape)==4, f"No. of dim in inp not equal to 4, got {inp.shape}"
    # no. of chanels in kernel and that in inp it should be same
    assert inp.shape[-1] == kernel.shape[-1], f"Mismatch in no. of channels in inp and kernel, got inp {inp.shape[-1]}, kernel {kernel.shape[-1]}"
    # non-square kernels are not allowed
    assert kernel.shape[0] == kernel.shape[1], f"dim 0 of kernel doesn't match dim 1, got {kernel.shape}"
    # inp shape square
    assert inp.shape[1] == inp.shape[2], f"Inp map dim 1 != dim 2, got {inp.shape}"
    
    # flip the kernel
    kernel = _rotate(kernel)
    
    oup = []
    start_rloc = 0
    end_rloc = kernel.shape[0]
    while end_rloc <= inp.shape[1]:
        output = []
        start_cloc = 0
        end_cloc = kernel.shape[1]
        while end_cloc <= inp.shape[2]:
            conv = (inp[:,start_rloc:end_rloc, start_cloc:end_cloc]*kernel).sum(axis=(1,2,3))
            output.append(conv)
            
            start_cloc += stride
            end_cloc += stride
        oup.append(output)
        start_rloc += stride
        end_rloc += stride
    return np.moveaxis(oup, -1, 0)
        
def _pad(inp, pad_width):   
    assert inp.shape[1] == inp.shape[2], f"Inp map dim 1 != dim 2, got {inp.shape}"
    return np.pad(inp, ((0,0), (pad_width,pad_width), (pad_width,pad_width), (0,0)))

def _inside_pad(inp, pad_width):
    assert inp.shape[1] == inp.shape[2], f"Inp map dim 1 != dim 2, got {inp.shape}"
    ix = np.repeat(np.arange(1, inp.shape[1]), pad_width)
    inp = np.insert(inp, ix, 0, axis=1)
    return np.insert(inp, ix, 0, axis=2)
